In [1]:
!pip install gluonts
!pip install --upgrade datasets
!pip install utilsforecast

  Using cached numpy-2.1.3-cp310-cp310-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.1.3-cp310-cp310-win_amd64.whl (12.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6


  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
claspy 0.2.1 requires numpy<1.25,>=1.21.0, but you have numpy 2.1.3 which is incompatible.
claspy 0.2.1 requires pandas<1.6.0,>=1.1.0, but you have pandas 2.2.3 which is incompatible.
claspy 0.2.1 requires scikit-learn<1.3.0,>=0.24.0, but you have scikit-learn 1.6.1 which is incompatible.
cvxpy 1.3.1 requires setuptools>65.5.1, but you have setuptools 59.5.0 which is incompatible.
mbrl 0.2.0 requires hydra-core==1.0.3, but you have hydra-core 1.3.2 which is incompatible.
mbrl 0.2.0 requires omegaconf<2.1,>=2.0, but you have omegaconf 2.3.0 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 2.1.3 which is incompatible.
plotly-calplot 0.1.16 requires numpy<2.0.0,>=1.22.3, but you have numpy 2.1.3 which is incomp

In [2]:
!git clone https://github.com/GiuliaGhisolfi/TSFM-ZeroShotEval
%cd TSFM-ZeroShotEval/src

c:\Documenti\VS_Code\tesi\TSFM-ZeroShotEval\src\TSFM-ZeroShotEval\src


Cloning into 'TSFM-ZeroShotEval'...
Updating files:  13% (20/153)
Updating files:  14% (22/153)
Updating files:  15% (23/153)
Updating files:  16% (25/153)
Updating files:  17% (27/153)
Updating files:  18% (28/153)
Updating files:  19% (30/153)
Updating files:  20% (31/153)
Updating files:  21% (33/153)
Updating files:  22% (34/153)
Updating files:  23% (36/153)
Updating files:  24% (37/153)
Updating files:  25% (39/153)
Updating files:  26% (40/153)
Updating files:  27% (42/153)
Updating files:  28% (43/153)
Updating files:  29% (45/153)
Updating files:  30% (46/153)
Updating files:  31% (48/153)
Updating files:  32% (49/153)
Updating files:  33% (51/153)
Updating files:  34% (53/153)
Updating files:  35% (54/153)
Updating files:  36% (56/153)
Updating files:  37% (57/153)
Updating files:  38% (59/153)
Updating files:  39% (60/153)
Updating files:  40% (62/153)
Updating files:  41% (63/153)
Updating files:  42% (65/153)
Updating files:  43% (66/153)
Updating files:  44% (68/153)
Upda

### Import data

In [3]:
import json
with open("data/chronos_data_proprieties.json") as f:
    dataset_properties_map = json.load(f)

In [4]:
DATASET_CHRONOS = ["exchange_rate", "ercot", "dominick"]

### Results file

In [5]:
import csv
import os

# Ensure the output directory exists
output_dir = "results"
os.makedirs(output_dir, exist_ok=True)

# Define the path for the CSV file
csv_file_path = os.path.join(output_dir, "chronos_data_results.csv")

with open(csv_file_path, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)

    # Write the header
    writer.writerow(
        [
            "dataset",
            "model",
            "eval_metrics/MSE[mean]",
            "eval_metrics/MSE[0.5]",
            "eval_metrics/MAE[0.5]",
            "eval_metrics/MASE[0.5]",
            "eval_metrics/MAPE[0.5]",
            "eval_metrics/sMAPE[0.5]",
            "eval_metrics/MSIS",
            "eval_metrics/RMSE[mean]",
            "eval_metrics/NRMSE[mean]",
            "eval_metrics/ND[0.5]",
            "eval_metrics/mean_weighted_sum_quantile_loss",
            "domain",
            "num_variates",
        ]
    )

## Evaluation

In [6]:
from gluonts.ev.metrics import (
    MSE,
    MAE,
    MASE,
    MAPE,
    SMAPE,
    MSIS,
    RMSE,
    NRMSE,
    ND,
    MeanWeightedSumQuantileLoss,
)

# Instantiate the metrics
metrics = [
    MSE(forecast_type="mean"),
    MSE(forecast_type=0.5),
    MAE(),
    MASE(),
    MAPE(),
    SMAPE(),
    MSIS(),
    RMSE(),
    NRMSE(),
    ND(),
    MeanWeightedSumQuantileLoss(
        quantile_levels=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    ),
]

### Chronos

In [7]:
model_name = "chronos_bolt_small" # TODO: change to "chronos_t5_base" for the original Chronos model

model_path="amazon/chronos-bolt-small",
# TODO: use "amazon/chronos-t5-base" for the corresponding original Chronos model
# "amazon/chronos-bolt-tiny", "amazon/chronos-bolt-mini", "amazon/chronos-bolt-small", "amazon/chronos-bolt-base",
# "amazon/chronos-t5-tiny", "amazon/chronos-t5-mini", "amazon/chronos-t5-small",
# "amazon/chronos-t5-base", "amazon/chronos-t5-large",

In [ ]:
from gluonts.dataset.split import TestData, OffsetSplitter

In [ ]:
from utils.chronos_predictor import ChronosPredictor
from gluonts.model import evaluate_model
from gluonts.time_feature import get_seasonality
import pandas as pd


for ds_name in DATASET_CHRONOS[0]:
        print(f"Processing dataset: {ds_name}")
        terms = ["short", "medium", "long"]
        for term in terms:
            ds_config = f"{ds_name}/{term}"

            dataset = pd.read_parquet("data/chronos_benchmark/exchange_rate.arrow")
            dataset = TestData( # TODO: change
                dataset=dataset,
                splitter=OffsetSplitter(offset=-8),
                prediction_length=8,
                windows=15,
                distance=None,
                max_history=None
            )

            season_length = get_seasonality(dataset_properties_map[ds_name]["freq"])
            prediction_length = dataset_properties_map[ds_name]["prediction_length"]

            print(f"Dataset size: {len(dataset.test_data)}")
            model = ChronosPredictor(
                model_path=model_path,
                num_samples=20,
                prediction_length=dataset.prediction_length,
                # Change device_map to "cpu" to run on CPU or "cuda" to run on GPU
                device_map="cpu",
            )
            predictor = model.create_predictor(batch_size=32)

            res = evaluate_model(
                predictor,
                test_data=dataset.test_data,
                metrics=metrics,
                batch_size=512,
                axis=None,
                mask_invalid_label=True,
                allow_nan_forecast=False,
                seasonality=season_length,
            )

            # Append the results to the CSV file
            with open(csv_file_path, "a", newline="") as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(
                    [
                        ds_name,
                        model_name,
                        res["MSE[mean]"][0],
                        res["MSE[0.5]"][0],
                        res["MAE[0.5]"][0],
                        res["MASE[0.5]"][0],
                        res["MAPE[0.5]"][0],
                        res["sMAPE[0.5]"][0],
                        res["MSIS"][0],
                        res["RMSE[mean]"][0],
                        res["NRMSE[mean]"][0],
                        res["ND[0.5]"][0],
                        res["mean_weighted_sum_quantile_loss"][0],
                        ds_name,
                        dataset_properties_map[ds_name]["num_variates"],
                    ]
                )

        print(f"Results for {ds_name} have been written to {csv_file_path}")